<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EB%82%A8%EC%83%81%EC%9A%B0_0926.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.4 MB/s eta 0:00:00


In [2]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=22f0e5b3a7d4f77aa35b1eafa3fee3023e375db1973be84f8cfa0f5ea088f8bd
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import torch
import torch.nn as nn
from torch import optim
from torch import optim

import warnings
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings("ignore")
%matplotlib inline

In [6]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/project_final/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 데이터 불러오기
- 맞춤법 완료
- 긍정 9000개, 부정 9000개 (골고루 섞었음)
- 단문 텍스트(리뷰 및 답변) 제거 완료
- 가게명, 요리명 삭제 완료

In [35]:
train = pd.read_csv(f"{DATA_PATH}train_0926_nsw.csv")
train

,review,reply,sentiment
0,"맛이 별로였어요, 조리 상태도 별로였고 배달 시간도 오래 걸렸네요","안녕하세요~! 맛이 별로였다고 하셨는데, 제 기준에는 싱겁고 딱히 맛있어 보이지 않...",0
1,양도 많고 맛나요,하핫 정말 이런 이런 칭찬에 춤추고 싶네요 든든한 생각나실 때 또 만나요 친절한 고...,1
2,음식의 퀄리티가 낮아서 별로였어요. 맛이 없어서 실망했어요,음식의 퀄리티가 낮아서 실망하셨다니 정말 죄송합니다. 더 맛있는 음식으로 만족스러운...,0
3,리뷰가 좋아서 주문했는데 정말 맛있게 잘 먹었습니다 다 맛있어요 김밥 완전 빅 사이...,감사합니다 언제나 푸짐한 양과 맛있는 맛으로 찾아뵙겠습니다 자주 애용해 주세요,1
4,항상 맛있게 잘 먹고 있습니다,고객님 소중한 고객님이 이임 항상 찾아주시고 맛있게 드셔주시니 너어어어무 기쁘고 행...,1
...,...,...,...
18405,친구시켜줬는데 사진은 이렇지만 고기 많고 맛있대요 감사합니다,저희 매장을 빛내 주셔서 영광 맛깔나는 사진 너무 감사해요 맛있게 드셔 주시기만 해...,1
18406,맛있게 먹는 단골집,자주자주 찾아주세요 저희 이용해 주셔서 감사합니다 최고보다는 최선을 다하는 될 수 ...,1
18407,가격이 좀 비싸서 아쉬웠어요. 좀 더 저렴하면 좋겠어요,가격이 비싸셨다니 정말 죄송합니다. 고객님의 의견을 반영하여 가격을 조정할 수 있도...,0
18408,센스가 없는 배달이었어요.,다음에는 더욱더 만족스러운 서비스를 제공하기 위해 노력하겠습니다.,0


# 모델 불러오기

In [ ]:
model_name = "skt/kogpt2-base-v2"

In [ ]:
# AutoModelForCausalLM 인과 관련 사용
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

# 데이터셋 만들기

In [ ]:
train.head()

,review,reply,sentiment
0,"맛이 별로였어요, 조리 상태도 별로였고 배달 시간도 오래 걸렸네요","안녕하세요~! 맛이 별로였다고 하셨는데, 제 기준에는 싱겁고 딱히 맛있어 보이지 않...",0
1,양도 많고 맛나요,하핫 정말 이런 이런 칭찬에 춤추고 싶네요 든든한 생각나실 때 또 만나요 친절한 고...,1
2,음식의 퀄리티가 낮아서 별로였어요. 맛이 없어서 실망했어요,음식의 퀄리티가 낮아서 실망하셨다니 정말 죄송합니다. 더 맛있는 음식으로 만족스러운...,0
3,리뷰가 좋아서 주문했는데 정말 맛있게 잘 먹었습니다 다 맛있어요 김밥 완전 빅 사이...,감사합니다 언제나 푸짐한 양과 맛있는 맛으로 찾아뵙겠습니다 자주 애용해 주세요,1
4,항상 맛있게 잘 먹고 있습니다,고객님 소중한 고객님이 이임 항상 찾아주시고 맛있게 드셔주시니 너어어어무 기쁘고 행...,1


In [ ]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        sentiment = item["sentiment"]
        review = item["review"]
        answer = item["reply"]

        return {
            "sentiment": sentiment,
            "review": review,
            "answer": answer
        }

In [ ]:
def collate_fn(batch):
    sentiment = torch.tensor([item["sentiment"] for item in batch])
    texts = [f"<q>{item['review']}</s><a>{item['answer']}</s>" for item in batch]
    x = tokenizer(texts, return_tensors="pt", padding=True)

    return {"x": x, "sentiment": sentiment}

In [ ]:
dt = ChatDataset(train)
dt[0]

{'sentiment': 0,
 'review': '맛이 별로였어요, 조리 상태도 별로였고 배달 시간도 오래 걸렸네요 ',
 'answer': '안녕하세요~! 맛이 별로였다고 하셨는데, 제 기준에는 싱겁고 딱히 맛있어 보이지 않았다고 하셨네요. 저희 메뉴의 맛에 대한 평가는 개인의 취향에 따라 달라질 수 있습니다만, 여러분의 의견에 감사드리며 앞으로 더욱 개선해 나가겠습니다! '}

In [ ]:
dl = torch.utils.data.DataLoader(dt,batch_size=2,collate_fn=collate_fn)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[ 9724,   455,   405, 23373, 15126,  8041,  8006, 13704, 13676, 10008,
           7235, 15126, 19570, 28005, 10135,  7235, 10714,  9539,  7422,  7098,
           8084,   739,     1,  9724,   439,   405,  7967,  7114,  8702,  7801,
           8084,   468,   376, 15882, 15126, 19132,  9078,  7816,  9287,  9037,
          10081,  9052, 16565, 36539, 16317,  8811, 11355, 49067, 14233, 30160,
           9078,  7816,  7098, 25856,  9265,  8806,  9803,  7155,  8143, 11355,
           8022,  9167, 41896, 12695, 46164,  9161, 27852,  9025,  9019, 16913,
           7182, 23636,  9311, 11926, 11955,  8022, 15940, 12854,  7530, 12634,
           9947, 12003,  8711, 12312,  6872, 16913,  7182,   376,   739,     1],
         [ 9724,   455,   405, 28922, 15394, 11355,  7055,  8084,   739,     1,
           9724,   439,   405,  8702,  8709, 29205, 10165, 10165, 19814,  8022,
          38725,  6889, 13358,  7098,  8084, 14309,  7283,  8704,  9658,  7055,
           7892,  90

In [ ]:
model(**batch["x"]).logits.shape

torch.Size([2, 90, 51200])

In [ ]:
batch["sentiment"].to(device, dtype=torch.float32)

tensor([0., 1.], device='cuda:0')

# 학습루프

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):

        # x = {key: val.to(device, dtype=torch.float32) for key, val in batch["x"].items()}
        # sentiment = batch["sentiment"].to(device, dtype=torch.float32)
        x = batch["x"].to(device)
        sentiment = batch["sentiment"].to(device)

        # batch["x"]["input_ids"].requires_grad = True
        # batch["x"]["attention_mask"].requires_grad = True

        pred = model(**x).logits  # batch, seq, 단어별 실수값
        n_class = pred.shape[-1]  # 클래스 개수
        sentiment_pred = pred[:, -1, :]

        pred = pred[:, :-1, :]  # eos 토큰 제외
        pred = pred.reshape(-1, n_class)  # 2차원 형태로 변환

        tgt = x["input_ids"][:, 1:]
        tgt = tgt.flatten()  # 1차원 벡터 형태로 바뀜

        mask = tgt != tokenizer.pad_token_id  # 패딩토큰 제외 -> 손실을 계산할 때 제외
        tgt = tgt[mask]
        pred = pred[mask]

        loss = loss_fn(pred, tgt)

        sentiment_loss = loss_fn(sentiment_pred, sentiment)

        optimizer.zero_grad()
        total_loss = loss + sentiment_loss
        total_loss.backward()
        optimizer.step()

        epoch_loss += total_loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

# 하이퍼파라미터 정의

In [ ]:
reset_seeds(SEED)
batch_size = 2
epochs = 10

model_name = "skt/kogpt2-base-v2"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
loss_fn = torch.nn.CrossEntropyLoss()

train_dt = ChatDataset(train)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for i in range(epochs):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
    print(f"{i+1}번째 epoch: {train_loss:.4f}")

# 학습모델 저장

In [ ]:
model.save_pretrained(f"{DATA_PATH}0926_nsw_loss_0.2")

# 저장한 모델 / 토크나이저 로드

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
loaded_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}0926_nsw_loss_0.2067").to(device)

In [ ]:
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

## 테스트루프

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score
bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def evaluate_similarity(input_text, generated_text, alpha=0.9): # cosine 가중치 조절
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))

    # 가중 평균(코사인 90% + 자카드 10%)
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

In [ ]:
evaluate_similarity("맛은 있는데 양이 적어요",'맛있게 드셔주셔서 감사합니다. 다음에 양 많이 드릴게요')

0.35823020339012146

In [ ]:
# # 옵션
# from transformers import pipeline
# pipe = pipeline("text2text-generation", model="lcw99/t5-base-korean-paraphrase")

# def paraphrase_text(text):
#     generated_text = pipe(text, max_length=512, early_stopping=True,
#                           num_beams=4,
#                           num_return_sequences=2,
#                           no_repeat_ngram_size=4,
#                           top_k=50,
#                           top_p=0.95)
#     generated_text = generated_text[0]['generated_text']
#     return generated_text

In [ ]:
def chatbot_with_evaluation(model, tokenizer, max_len, device, num_samples=5):
    model.eval()
    while True:
        text = input("user > ").strip()
        if text == "quit":
            break

        text = "<q>" + text + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        best_generated_text = None
        best_similarity_score = -1.0
        generated_texts = []  # 답변 후보군을 담을 리스트
        for i in range(num_samples):
            result_ids = model.generate(x,
                                        max_length=max_len,
                                        repetition_penalty=2.0,
                                        num_beams=1,
                                        num_return_sequences=1,
                                        no_repeat_ngram_size=4,
                                        use_cache=True,
                                        do_sample=True,
                                        temperature=0.8,
                                        top_k=50,
                                        )
            generated_text = tokenizer.decode(result_ids[0])
            generated_text = generated_text[q_len:-4]
            generated_text = re.sub(r'[^가-힣]', ' ', generated_text)

            similarity_score = evaluate_similarity(text, generated_text)
            generated_texts.append((similarity_score,generated_text))
            print(generated_texts[i])

            if similarity_score > best_similarity_score:
                best_similarity_score = similarity_score
                best_generated_text = generated_text

        print()
        print("Best_reply >", best_generated_text)
        print("Best_similarity_score:", best_similarity_score)
        print("--------------------------------------------------------------------------------------------")


In [ ]:
pd.read_csv(f"{DATA_PATH}SSA.csv").head(3)

,input_text
0,재료를 좋은걸 쓰시나봐요! 맛이 좋습니다.
1,서비스로 주신 감자튀김이 너무 맛있어서 다음에 주문하려고요.
2,우리 동네에 이런 맛집이 있다니 앞으로 자주 시켜먹겠습니다.


In [ ]:
ssa_df = pd.read_csv(f"{DATA_PATH}SSA.csv")

In [ ]:
# SSA 평가를 위해 df 형태로 출력
def evaluation_df(df, model, tokenizer, max_len, device, num_samples=5):
    model.eval()
    output_text = []
    for text in tqdm(df['input_text']):
        text = "<q>" + text + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        best_generated_text = None
        best_similarity_score = -1.0
        generated_texts = []  # 답변 후보군을 담을 리스트
        for i in range(num_samples):
            result_ids = model.generate(x,
                                        max_length=max_len,
                                        repetition_penalty=2.0,
                                        num_beams=1,
                                        num_return_sequences=1,
                                        no_repeat_ngram_size=4,
                                        use_cache=True,
                                        do_sample=True,
                                        temperature=0.8,
                                        top_k=50,
                                        )
            generated_text = tokenizer.decode(result_ids[0])
            generated_text = generated_text[q_len:-4]
            generated_text = re.sub(r'[^가-힣]', ' ', generated_text)

            similarity_score = evaluate_similarity(text, generated_text)
            generated_texts.append((similarity_score,generated_text))

            if similarity_score > best_similarity_score:
                best_similarity_score = similarity_score
                best_generated_text = generated_text

        output_text.append(best_generated_text)
    df['output_text'] = output_text
    return df


In [ ]:
# 0922_확정 모델
loaded_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}sohyun_23762_31_loss_0.2067").to(device)
df = evaluation_df(ssa_df,loaded_model,tokenizer,128,device)
df

  0%|          | 0/49 [00:00<?, ?it/s]

,input_text,output_text
0,재료를 좋은걸 쓰시나봐요! 맛이 좋습니다.,안녕하세요 저희 수제 죽 전문점입니다 다른 곳에 비해 가격이 비싼 편지만 맛있는...
1,서비스로 주신 감자튀김이 너무 맛있어서 다음에 주문하려고요.,주문 감사합니다 맛이 기대만큼 나오지 않아서 실망스러우셨을 것 같아요 봅니다 더...
2,우리 동네에 이런 맛집이 있다니 앞으로 자주 시켜먹겠습니다.,고객님 감사합니다 항상 이용해 주시고요 다음에도 좋은 모습으로 찾아뵐게요입니다...
3,맛있고 양도 많아서 만족스럽게 식사했습니다. 번창하세요~,안녕 하오랑 친구분들이 맛있게 드셨다니 정말 뿌듯합니다 앞으로도 맛있는 음식으로 보...
4,샌드위치 빵도 촉촉하고 내용 구성도 너무 좋아요.,감사합니다 고객님 같은 메뉴로 좋은 시간 보내셨을까요 어떤 부분이 가장 기본인데...
5,고기가 많이 타서 아쉽네요.,저희 고기 맛에 문제가 있었다니 정말 죄송합니다 제대로 구워지지 않은 것 같아 안...
6,생각했던 것보다 양이 적어서 부족했습니다.,안녕하세요 항상 양을 입장에서 생각합니다 저희 음식이 맞지 않으셨다니 죄송합니다...
7,배달이 늦게 와서 피자가 다 식어서 맛이 없었어요.,서비스 했을 때 음식의 신선도와 품질이 저조했던 것 같아 죄송합니다 이번에 발생한...
8,친구가 추천해서 기대를 많이 했는데 저는 맛이 없네요. 왜 유명한지 모르겠어요.,고객님께서 말씀해주신 점에 대해 전혀 감사드립니다 누구이에서 다른 맛을 느끼셨다고 ...
9,너무 비싸서 자주 못 시켜먹겠어요.,가격 대비로는 음식의 질은 좋았지만 음식물이 만족스럽지 않았나봅니다 더 합리적인 ...


In [ ]:
df.to_csv('df.csv', index=False, encoding = "utf-8-sig")

# 엑셀로 확인
with pd.ExcelWriter("df.xlsx", engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='시트1', index=False)

In [ ]:
# 0925_확정 모델
loaded_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}sohyun_23762_99_loss_0.1528").to(device)
chatbot_with_evaluation(loaded_model,tokenizer,128,device)